In [3]:
mode040_fnames = io.l1b_filenames('*IPH2-*-mode040-muv*',stage=False)
mode040_fnames.sort()
print(len(mode040_fnames))
mode040_fnames[:5]

200


['/maven_iuvs/production/products/level1b/mvn_iuv_l1b_IPH2-cycle00001-mode040-muv_20140419T004426_v00_r00.fits.gz',
 '/maven_iuvs/production/products/level1b/mvn_iuv_l1b_IPH2-cycle00002-mode040-muv_20140419T084429_v00_r00.fits.gz',
 '/maven_iuvs/production/products/level1b/mvn_iuv_l1b_IPH2-cycle00003-mode040-muv_20140419T164430_v00_r00.fits.gz',
 '/maven_iuvs/production/products/level1b/mvn_iuv_l1b_IPH2-cycle00004-mode040-muv_20140420T004433_v00_r00.fits.gz',
 '/maven_iuvs/production/products/level1b/mvn_iuv_l1b_IPH2-cycle00005-mode040-muv_20140420T084434_v00_r00.fits.gz']

In [4]:
from IPython.parallel import Client
c = Client()

In [5]:
all_engines=c[:]
lb = c.load_balanced_view()

In [6]:
%%px
import os
from iuvs import io
import numpy as np

In [48]:
import os
from iuvs import io

from skimage.exposure import rescale_intensity

def imshow_and_cbar(data, ax, txt, vmin=None, vmax=None):
    im = ax.imshow(data, vmin=vmin, vmax=vmax)
    cbar = plt.colorbar(im, ax=ax)
    cbar.ax.tick_params(labelsize=9)
    ax.set_title(txt)
    
def generate_dark_overview(fname, interactive=False, data=None):
    from iuvs import io
    fig, axes = plt.subplots(2,2)
    iuvsfname = io.Filename(fname)
    axes = axes.flatten()
    if not data:
        data = io.L1BReader(fname)
    subbed_img = data.detector_background_subtracted[:850]
    dark = data.detector_dark[:850]
    p2, p98 = np.percentile(subbed_img, (2, 98))
    imshow_and_cbar(subbed_img, axes[0], 'subtracted, 2/98-stretch',
                   vmin=p2, vmax=p98)
    axes[1].hist(subbed_img.flatten(), bins=100, range=(-500,500),
                 log=True)
    axes[1].set_xlim(-600,600)
    axes[1].set_ylim(1, 10**6)
    p2, p98 = np.percentile(dark, (2, 98))
    imshow_and_cbar(dark, axes[2], 'dark, 2/98-stretch',
                    vmin=p2, vmax=p98)
    axes[3].hist(dark.flatten(), bins=100, log=True, range=(0,1000))
    axes[3].set_xlim(-100, 1100)
    axes[3].set_ylim(1, 10**6)
    fig.suptitle('Exposure: {}, Time:{}'.format(data.img_header['INT_TIME'],
                                              iuvsfname.time.isoformat()))
    plt.savefig(os.environ['HOME']+'/public_html/level1b_darks/plots/l1b_img_vs_darks_{}'.format(iuvsfname.time.isoformat()),
            dpi=150)
    if not interactive:
        plt.close(fig)
        return iuvsfname.basename
    else:
        return data

In [8]:
%matplotlib qt

In [49]:
data = generate_dark_overview(mode040_fnames[0], interactive=True,
                              data=data)

In [50]:
all_engines.push(dict(
        imshow_and_cbar=imshow_and_cbar,
        generate_dark_overview=generate_dark_overview,
))

<AsyncResult: finished>

In [51]:
todo = mode040_fnames[:]
results = lb.map_async(generate_dark_overview, todo)

In [52]:
import time
while not results.ready():
    print(round(100*results.progress/len(todo)), ' %')
    time.sleep(10)

0  %
0  %
5  %
10  %
15  %
16  %
20  %
25  %
30  %
34  %
37  %
40  %
45  %
50  %
52  %
56  %
60  %
65  %
70  %
74  %
77  %
81  %
85  %
89  %
92  %
96  %


In [53]:
%%bash
source activate sigal
cd ~/public_html/level1b_darks
./do_gallery.sh

Sorting media
Processing files

Done.
Processed 200 images and 0 videos in 9.51 seconds.


discarding /usr/local/python3/miniconda/bin from PATH
prepending /usr/local/python3/miniconda/envs/sigal/bin to PATH
